In [1]:
ROOT = './ODIR-5K-2'

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision
from sklearn.metrics import roc_auc_score
BATCH_SIZE = 32
IMG_HEIGHT = 250
IMG_WIDTH = 250
CLASS_NAMES = ['D', 'N']

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = ImageFolder(root=ROOT+'/Training_Images/', transform=transform)
# validation_set = ImageFolder(root=ROOT+'/Validation_Images/', transform=transform)
test_set = ImageFolder(root=ROOT+'/Testing_Images/', transform=transform)

In [9]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size = 32,
    shuffle = True,
    num_workers = 2
)

# validation_loader = torch.utils.data.DataLoader(
#     validation_set,
#     batch_size = 16,
#     shuffle = False,
#     num_workers = 2
# )

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size = 16,
    shuffle = False,
    num_workers = 2
)

In [10]:
def train_model(model, num_epochs, criterion, optimizer, results_path):
    model.train()
    train_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)
    train_accracy = np.zeros(num_epochs)
    val_accracy = np.zeros(num_epochs)

    for epoch in range(num_epochs):
        # trainning
        running_loss = 0.0
        n = 0
        total=0
        correct=0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += 1
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)    # add in the number of labels in this minibatch
            correct += (predicted == labels).sum().item()  # add in the number of correct labels
        train_losses[epoch] = running_loss / n
        train_accracy[epoch]=correct/total
        # validation
#         running_loss = 0.0
#         n = 0
#         total=0
#         correct=0
#         with torch.no_grad():
#             for images,labels in validation_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs=model(images)
#                 running_loss += criterion(outputs,labels).item()
#                 n += 1
#                 _, predicted = torch.max(outputs.data, 1)
#                 total += labels.size(0)    # add in the number of labels in this minibatch
#                 correct += (predicted == labels).sum().item()  # add in the number of correct labels
#         val_losses[epoch]=running_loss/n
#         val_accracy[epoch]=correct/total

        print(f'Epoch [{epoch + 1}/{num_epochs}], training loss: {train_losses[epoch] : .3f} training accuracy: {train_accracy[epoch]: .1%}')

    torch.save({"state_dict": model.state_dict(), "train_losses": train_losses, "train_accracy": train_accracy}, results_path)


In [21]:
model_vgg16 = torchvision.models.vgg16(pretrained=False)
in_features = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(in_features, len(CLASS_NAMES), True)
model_vgg16 = model_vgg16.to(device)

result_path = ROOT+'/results/model_vgg_binary.pt'    
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_vgg16.parameters(), lr=0.01)
num_epochs = 20

train_model(model_vgg16, num_epochs, criterion, optimizer, result_path)

Epoch [1/20], training loss:  42036195404.031 training accuracy:  50.3%
Epoch [2/20], training loss:  7.822 training accuracy:  49.6%
Epoch [3/20], training loss:  0.881 training accuracy:  50.2%
Epoch [4/20], training loss:  1.079 training accuracy:  50.2%
Epoch [5/20], training loss:  0.700 training accuracy:  50.8%
Epoch [6/20], training loss:  0.698 training accuracy:  49.5%
Epoch [7/20], training loss:  0.697 training accuracy:  50.4%
Epoch [8/20], training loss:  0.699 training accuracy:  50.6%
Epoch [9/20], training loss:  0.695 training accuracy:  50.1%
Epoch [10/20], training loss:  0.695 training accuracy:  50.2%
Epoch [11/20], training loss:  0.695 training accuracy:  49.7%
Epoch [12/20], training loss:  0.694 training accuracy:  50.9%
Epoch [13/20], training loss:  0.694 training accuracy:  50.4%
Epoch [14/20], training loss:  0.694 training accuracy:  50.7%
Epoch [15/20], training loss:  0.694 training accuracy:  49.6%
Epoch [16/20], training loss:  0.694 training accuracy

Define LeNet-5 model.

In [13]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # 3 input image channels, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(55696, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(CLASS_NAMES))

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Train LeNet-5 model

In [25]:
# 初始化模型、损失函数和优化器
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.0001)
results_path = ROOT + '/results/LeNet5_BINARY.pt'
num_epochs = 20

train_model(model, num_epochs, criterion, optimizer, results_path)

Epoch [1/20], training loss:  0.978 training accuracy:  49.0%
Epoch [2/20], training loss:  0.693 training accuracy:  50.6%
Epoch [3/20], training loss:  0.693 training accuracy:  50.8%
Epoch [4/20], training loss:  0.694 training accuracy:  50.8%
Epoch [5/20], training loss:  0.693 training accuracy:  50.8%
Epoch [6/20], training loss:  0.694 training accuracy:  49.0%
Epoch [7/20], training loss:  0.694 training accuracy:  49.8%
Epoch [8/20], training loss:  0.693 training accuracy:  49.8%
Epoch [9/20], training loss:  0.693 training accuracy:  49.7%
Epoch [10/20], training loss:  0.694 training accuracy:  50.1%
Epoch [11/20], training loss:  0.693 training accuracy:  51.4%
Epoch [12/20], training loss:  0.694 training accuracy:  50.8%
Epoch [13/20], training loss:  0.693 training accuracy:  50.8%
Epoch [14/20], training loss:  0.693 training accuracy:  49.8%
Epoch [15/20], training loss:  0.693 training accuracy:  49.7%
Epoch [16/20], training loss:  0.693 training accuracy:  50.8%
E

In [31]:
resnet18 = torchvision.models.resnet18(pretrained=False)
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, len(CLASS_NAMES), True)
resnet18 = resnet18.to(device)

result_path = ROOT+'/results/model_resnet18_binary.pt'    
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.01)
num_epochs = 20

train_model(resnet18, num_epochs, criterion, optimizer, result_path)

Epoch [1/20], training loss:  0.598 training accuracy:  71.9%
Epoch [2/20], training loss:  0.589 training accuracy:  71.5%
Epoch [3/20], training loss:  0.511 training accuracy:  74.2%
Epoch [4/20], training loss:  0.494 training accuracy:  74.7%
Epoch [5/20], training loss:  0.478 training accuracy:  75.2%
Epoch [6/20], training loss:  0.476 training accuracy:  75.0%
Epoch [7/20], training loss:  0.480 training accuracy:  75.2%
Epoch [8/20], training loss:  0.494 training accuracy:  74.2%
Epoch [9/20], training loss:  0.474 training accuracy:  75.2%
Epoch [10/20], training loss:  0.479 training accuracy:  75.3%
Epoch [11/20], training loss:  0.484 training accuracy:  74.9%
Epoch [12/20], training loss:  0.471 training accuracy:  75.3%
Epoch [13/20], training loss:  0.473 training accuracy:  75.3%
Epoch [14/20], training loss:  0.472 training accuracy:  75.2%
Epoch [15/20], training loss:  0.468 training accuracy:  75.3%
Epoch [16/20], training loss:  0.467 training accuracy:  75.4%
E

In [15]:
def test_model(model, path):
    d = torch.load(path)
    model.load_state_dict(d["state_dict"]) 
    model = model.to(device)
    total = 0
    correct = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    prob_all = []
    label_all = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Append predicted probabilities and labels to the lists
            prob_all.extend(outputs[:, 1].cpu().numpy())
            label_all.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            true_positives += ((predicted == 1) & (labels == 1)).sum().item()
            false_positives += ((predicted == 1) & (labels == 0)).sum().item()
            false_negatives += ((predicted == 0) & (labels == 1)).sum().item()

    accuracy = correct / total
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    # Convert the lists to numpy arrays
    label_all = np.array(label_all)
    prob_all = np.array(prob_all)

    AUC = roc_auc_score(label_all, prob_all)

    return accuracy, precision, recall, AUC


In [21]:
results_path = ROOT + '/results/LeNet5_BINARY.pt'
model = LeNet5()
accuracy, precision, recall, AUC = test_model(model, results_path)

print('leNet5:')
print('  accuracy: {accuracy:.2f}'.format(accuracy=accuracy))
print('  precision: {precision:.2f}'.format(precision=precision))
print('  recall: {recall:.2f}'.format(recall=recall))
print('  AUC: {AUC:.2f}'.format(AUC=AUC))

leNet5:
  accuracy: 0.66
  precision: 0.66
  recall: 1.00
  AUC: 0.50


In [23]:
model_vgg16 = torchvision.models.vgg16(pretrained=False)
in_features = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(in_features, len(CLASS_NAMES), True)

results_path = ROOT + '/results/model_vgg_binary.pt'

accuracy, precision, recall, AUC = test_model(model_vgg16, results_path)

print('VGG16:')
print('  accuracy: {accuracy:.2f}'.format(accuracy=accuracy))
print('  precision: {precision:.2f}'.format(precision=precision))
print('  recall: {recall:.2f}'.format(recall=recall))
print('  AUC: {AUC:.2f}'.format(AUC=AUC))

VGG16:
  accuracy: 0.54
  precision: 0.68
  recall: 0.58
  AUC: 0.53


In [24]:
resnet18 = torchvision.models.resnet18(pretrained=False)
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, len(CLASS_NAMES), True)

results_path = ROOT + '/results/model_resnet18_binary.pt'

accuracy, precision, recall, AUC = test_model(resnet18, results_path)

print('ResNet18:')
print('  accuracy: {accuracy:.2f}'.format(accuracy=accuracy))
print('  precision: {precision:.2f}'.format(precision=precision))
print('  recall: {recall:.2f}'.format(recall=recall))
print('  AUC: {AUC:.2f}'.format(AUC=AUC))

ResNet18:
  accuracy: 0.56
  precision: 0.65
  recall: 0.74
  AUC: 0.52
